# 简单RNN

In [4]:
import numpy as np

class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        # 初始化权重
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01   # 输入层到隐藏层的权重
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # 隐藏层到隐藏层的权重
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  # 隐藏层到输出层的权重
        
        # 初始化偏置
        self.bh = np.zeros((hidden_size, 1))  # 隐藏层偏置
        self.by = np.zeros((output_size, 1))  # 输出层偏置
        
    def forward(self, inputs, hidden_state):
        # 输入: inputs是一个序列，hidden_state是隐藏状态
        self.inputs = inputs
        self.hidden_states = []
        h = hidden_state
        
        # 前向传播过程
        for x in inputs:
            # 重塑输入为列向量
            x = x.reshape(-1, 1)
            
            # 计算隐藏状态
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)
            self.hidden_states.append(h)
        
        # 计算输出
        y = np.dot(self.Why, h) + self.by
        
        return y, h
    
    def backward(self, target, learning_rate=0.01):
        # 初始化梯度
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)
        
        # 获取最后的隐藏状态
        h = self.hidden_states[-1]
        
        # 计算输出层的误差
        # 均方损失函数中，损失函数对输出的导数是输出-目标值
        dy = self.output - target
        
        # 计算Why的梯度
        dWhy += np.dot(dy, h.T)
        dby += dy
        
        # 反向传播通过时间
        dh_next = np.zeros_like(h)
        
        for t in reversed(range(len(self.inputs))):
            # 当前时间步的隐藏状态
            h = self.hidden_states[t]
            
            # 计算隐藏层的梯度
            dh = np.dot(self.Why.T, dy) + dh_next
            
            # 计算tanh的导数
            dtanh = (1 - h * h) * dh
            
            # 计算各个权重的梯度
            dbh += dtanh
            dWxh += np.dot(dtanh, self.inputs[t].reshape(1, -1))
            dWhh += np.dot(dtanh, self.hidden_states[t-1].T) if t > 0 else 0
            
            # 计算传递给下一个时间步的梯度
            dh_next = np.dot(self.Whh.T, dtanh)
        
        # 更新权重和偏置
        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh -= learning_rate * dbh
        self.by -= learning_rate * dby

def main():
    # 创建训练数据
    sequence = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
    target = np.array([0.6])
    
    # 初始化RNN
    rnn = SimpleRNN(input_size=1, hidden_size=8, output_size=1)
    
    # 训练参数
    epochs = 1000
    learning_rate = 0.01
    
    # 训练循环
    for epoch in range(epochs):
        # 初始化隐藏状态
        hidden_state = np.zeros((8, 1))
        
        # 前向传播
        output, final_hidden = rnn.forward(sequence, hidden_state)
        
        # 计算损失
        loss = np.mean((output - target) ** 2)
        
        # 存储输出用于反向传播
        rnn.output = output
        
        # 反向传播
        rnn.backward(target, learning_rate)
        
        # 每100个epoch打印一次损失
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}')
    
    # 训练结束后进行预测
    hidden_state = np.zeros((8, 1))
    final_output, _ = rnn.forward(sequence, hidden_state)
    print("\n训练完成:")
    print("预测输出:", final_output)
    print("目标值:", target)
    print("最终损失:", np.mean((final_output - target) ** 2))

if __name__ == "__main__":
    main()

Epoch 0, Loss: 0.3603
Epoch 100, Loss: 0.0473
Epoch 200, Loss: 0.0060
Epoch 300, Loss: 0.0007
Epoch 400, Loss: 0.0001
Epoch 500, Loss: 0.0000
Epoch 600, Loss: 0.0000
Epoch 700, Loss: 0.0000
Epoch 800, Loss: 0.0000
Epoch 900, Loss: 0.0000

训练完成:
预测输出: [[0.59998252]]
目标值: [0.6]
最终损失: 3.055896246576464e-10
